### Overview of Changes to Data_Cleaning.ipynb

- Data Types mapped with category for higher cardinality instead of strings only for memory efficiency
- Moved datetime handling earlier in the process
- Created temporal and spatial features for next steps (forecasting and clustering)
- Forward filling for time series instead of dropping any rows with data missing

### Import

In [2]:
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

### Data Ingestion & Type Map

Category dtypes for memory usage efficiency

In [3]:
dtype_map = {
    'truck': 'category',  
    'square_id': 'category',
    'Transfer unit': 'string',
    'bitrate_unit': 'string'
}

file_paths = glob.glob('data/*-combined-kml.csv')
df = pd.concat(
    [pd.read_csv(p, dtype=dtype_map, low_memory=False) for p in file_paths],
    ignore_index=True
)

Selectively convert numeric columns. (Avoid converting datetime upfront.)

In [4]:
numeric_cols = [
    'latitude', 'longitude', 'speed', 
    'svr1', 'svr2', 'svr3', 'svr4',
    'Transfer size', 'Bitrate', 'Retransmissions', 'CWnd',
    'Transfer size-RX', 'Bitrate-RX', 'send_data'
]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

In [5]:
df_copy = df.copy()

In [6]:
df = df_copy.copy()

### Data Inspection

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3042495 entries, 0 to 3042494
Data columns (total 32 columns):
 #   Column            Dtype  
---  ------            -----  
 0   time              int64  
 1   Day               object 
 2   Year              float64
 3   Month             float64
 4   Date              float64
 5   hour              float64
 6   min               float64
 7   sec               float64
 8   timezone          object 
 9   latitude          float64
 10  longitude         float64
 11  speed             float64
 12  truck             object 
 13  svr1              float64
 14  svr2              float64
 15  svr3              float64
 16  svr4              float64
 17  Role              object 
 18  Transfer size     float64
 19  Transfer unit     string 
 20  Bitrate           float64
 21  bitrate_unit      string 
 22  Retransmissions   float64
 23  CWnd              float64
 24  cwnd_unit         object 
 25  Role-RX           object 
 26  Transfer size-

In [8]:
df.describe()

,time,Year,Month,Date,hour,min,sec,latitude,longitude,speed,...,svr2,svr3,svr4,Transfer size,Bitrate,Retransmissions,CWnd,Transfer size-RX,Bitrate-RX,send_data
count,3.042495e+06,2698689.0,2698689.0,2.698689e+06,2.698688e+06,2.698688e+06,2.698688e+06,2.698686e+06,2.698686e+06,2.698685e+06,...,2.448520e+06,2.448520e+06,2.448518e+06,2.713533e+06,2.713533e+06,2.711701e+06,2.711698e+06,2.713460e+06,2.713460e+06,2.788962e+06
mean,1.657619e+09,2022.0,7.0,1.218967e+01,8.817216e+00,2.961934e+01,2.948782e+01,-2.737269e+01,2.096419e+02,1.374754e+01,...,2.838819e+02,3.030290e+02,2.998318e+02,1.068046e+00,8.966052e+00,2.107719e+00,1.271308e-02,2.014400e+00,1.689644e+01,2.996928e-01
std,3.814960e+06,0.0,0.0,6.021038e+00,2.308625e+00,1.745151e+01,1.731720e+01,3.622062e+01,2.262437e+02,2.293579e+01,...,3.725267e+02,3.651250e+02,3.667629e+02,1.140621e+00,8.254232e+00,1.049226e+01,1.452743e-01,1.992016e+00,1.661119e+01,9.339412e-01
min,-2.147484e+09,2022.0,7.0,4.000000e+00,4.000000e+00,0.000000e+00,0.000000e+00,-3.782882e+01,1.447181e+02,-1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.657080e+09,2022.0,7.0,6.000000e+00,7.000000e+00,1.400000e+01,1.400000e+01,-3.778170e+01,1.447747e+02,2.852080e-01,...,3.340000e+01,3.810000e+01,5.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.460938e-01,6.260000e+00,0.000000e+00
50%,1.657592e+09,2022.0,7.0,1.200000e+01,9.000000e+00,3.000000e+01,2.900000e+01,-3.775104e+01,1.448013e+02,2.744664e+00,...,6.290000e+01,1.430000e+02,8.180000e+01,1.250000e+00,1.050000e+01,0.000000e+00,1.025391e-02,1.470000e+00,1.230000e+01,3.130817e-02
75%,1.658123e+09,2022.0,7.0,1.800000e+01,1.100000e+01,4.500000e+01,4.400000e+01,-3.771678e+01,1.448267e+02,1.592164e+01,...,4.120000e+02,4.270000e+02,4.340000e+02,1.250000e+00,1.050000e+01,1.000000e+00,1.025391e-02,2.900000e+00,2.430000e+01,2.629497e-01
max,1.658479e+09,2022.0,7.0,2.200000e+01,1.600000e+01,5.900000e+01,5.900000e+01,9.900000e+01,9.990000e+02,1.086105e+02,...,2.496000e+03,2.495000e+03,2.523000e+03,9.330000e+02,9.440000e+01,1.288000e+03,2.050000e+02,3.140000e+02,3.370000e+02,1.861712e+01


In [9]:
df.head()

,time,Day,Year,Month,Date,hour,min,sec,timezone,latitude,...,Retransmissions,CWnd,cwnd_unit,Role-RX,Transfer size-RX,Transfer unit-RX,Bitrate-RX,bitrate_unit-RX,send_data,square_id
0,1658274194,Wed,2022.0,7.0,20.0,9.0,43.0,14.0,AEST,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1658274194,Wed,2022.0,7.0,20.0,9.0,43.0,14.0,AEST,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1658274194,Wed,2022.0,7.0,20.0,9.0,43.0,14.0,AEST,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1658274194,Wed,2022.0,7.0,20.0,9.0,43.0,14.0,AEST,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1658274195,Wed,2022.0,7.0,20.0,9.0,43.0,15.0,AEST,99.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Datetime and Timezone Handling (Time-series)

In [10]:
df['datetime'] = pd.to_datetime(df['time'], unit='s', utc=True).dt.tz_convert('Australia/Melbourne')
df.drop(columns=['time'], inplace=True) 

### GPS Cleaning and Filtering

In [11]:
df[['latitude', 'longitude']] = df[['latitude', 'longitude']].replace(999, np.nan)
df.dropna(subset=['latitude', 'longitude'], inplace=True)

### Invalid Sensor Rows Removal

In [12]:
svr_cols = ['svr1', 'svr2', 'svr3', 'svr4']
other_cols = ['Transfer size', 'Bitrate', 'Retransmissions', 'CWnd', 
              'Transfer size-RX', 'Bitrate-RX', 'send_data']
mask = (df[svr_cols].eq(1000).all(axis=1)) & (df[other_cols].eq(0).all(axis=1))
df = df[~mask]

### Initial Feature Engineering for Clustering and Forecasting

In [13]:
df['hour'] = df['datetime'].dt.hour
df['day_of_week'] = df['datetime'].dt.dayofweek  # Monday=0, Sunday=6

In [14]:
spatial_agg = df.groupby('square_id').agg(
    avg_latency=('svr1', 'mean'),
    avg_bitrate=('Bitrate', 'mean'),
    total_retransmissions=('Retransmissions', 'sum'),
    total_upload=('send_data', 'sum'),
    latitude=('latitude', 'mean'),
    longitude=('longitude', 'mean')
).reset_index()

In [ ]:
hourly_df = df.set_index('datetime').resample('1h').agg({
    'svr1': 'mean',         # as a proxy for latency
    'Bitrate': 'mean',
    'Retransmissions': 'sum',
    'send_data': 'sum'
}).reset_index()

/tmp/ipykernel_17867/3416722886.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  hourly_df = df.set_index('datetime').resample('1H').agg({


In [16]:
hourly_df['bitrate_lag1'] = hourly_df['Bitrate'].shift(1) # lag feature
hourly_df.fillna(method='ffill', inplace=True) # forward fill, maintain continuity

/tmp/ipykernel_17867/4093412870.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hourly_df.fillna(method='ffill', inplace=True) # forward fill, maintain continuity


### Normalisation and Output

In [17]:
scaler = StandardScaler()
spatial_agg[['avg_latency', 'avg_bitrate']] = scaler.fit_transform(spatial_agg[['avg_latency', 'avg_bitrate']])

In [18]:
spatial_agg.head()

,square_id,avg_latency,avg_bitrate,total_retransmissions,total_upload,latitude,longitude
0,square_103079215125,-1.089433,NaN,0.0,21.394897,-37.731000,144.745058
1,square_103079215126,-0.656329,0.299258,1123.0,192.068449,-37.728893,144.749634
2,square_103079215128,0.705965,-1.539485,261.0,26.546056,-37.725008,144.759699
3,square_103079215130,1.035333,-0.350960,19256.0,416.306826,-37.727828,144.766274
4,square_103079215132,0.746213,-0.162983,24723.0,927.872513,-37.727791,144.772566


In [19]:
hourly_df.head()

,datetime,svr1,Bitrate,Retransmissions,send_data,bitrate_lag1
0,2022-07-04 05:00:00+10:00,164.761605,9.587114,19212.0,53.821966,NaN
1,2022-07-04 06:00:00+10:00,137.114572,9.674189,60355.0,172.982437,9.587114
2,2022-07-04 07:00:00+10:00,119.750291,9.124099,54765.0,105.946353,9.674189
3,2022-07-04 08:00:00+10:00,172.741844,8.571186,57025.0,127.174063,9.124099
4,2022-07-04 09:00:00+10:00,243.929796,8.165318,59450.0,128.502862,8.571186


In [ ]:
spatial_agg.to_csv('../data/spatial_clustering_data.csv', index=False)
hourly_df.to_csv('../data/time_series_forecasting_data.csv', index=False)